# 변수 엔지니어링

강의마다 수강생이 질문을 하였을 때 강사의 답변 유무에 따라 수강생 수 혹은 리뷰 점수 등 강의에 대한 긍정적 / 부정적 영향이 있는지 파악하기 위해 피처 엔지니어링을 진행합니다.

질문 게시판에서 스크래핑한 질문 데이터를 정제하여 강의별 총 질문 개수 / 강사가 질문에 답변한 개수 / 강사의 질문 답변률 / 강사가 질문에 답변까지 걸린 기간을 변수로 추출하여 'course_df'(강의 게시판에서 스크래핑한 강의 데이터)에 추가하는 과정입니다.

In [1]:
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime
import warnings

warnings.filterwarnings(action='ignore')

In [29]:
course_df = pd.read_csv('o2_course.csv', index_col = 0)
question_df = pd.read_csv('o2_question.csv', index_col = 0)

In [30]:
# 변수 생성에 필요한 컬럼들만 가져옴
temp_question_df = question_df.copy()
temp_question_df.drop(columns = ['title', 'q_content', 'a_content'], inplace = True)

In [31]:
# question_df에서 강의 제목을 course_df에서 찾아, 강의의 강사를 question_df로 가져옴
instructor_list = []

for question_course in temp_question_df['course_title'].values :
    instructor_list.append(course_df.instructor[course_df['course_title'] == question_course].tolist())

temp_question_df['instructor'] = instructor_list
temp_question_df['instructor'] = temp_question_df['instructor'].str[0]

In [32]:
# 질문에 대한 답변의 개수를 체크
answer_cnt_list = []

for answer_cnt in temp_question_df['answer'].values :
    answer_cnt_list.append(len(answer_cnt))

In [33]:
# 스크래핑에서 답변이 여러 개인 경우, 리스트 형태로 추출하였으므로 분석할 방식에 맞게 정제
temp_question_df['answer'] = temp_question_df['answer'].astype('str')
temp_question_df['instructor'] = temp_question_df['instructor'].astype('str')

temp_question_df['answer'] = temp_question_df['answer'].str.replace('[', '', regex = True)
temp_question_df['answer'] = temp_question_df['answer'].str.replace("'", '', regex = True)
temp_question_df['answer'] = temp_question_df['answer'].str.replace(']', '', regex = True)

temp_question_df['a_date'] = temp_question_df['a_date'].str.replace('[', '', regex = True)
temp_question_df['a_date'] = temp_question_df['a_date'].str.replace("'", '', regex = True)
temp_question_df['a_date'] = temp_question_df['a_date'].str.replace(']', '', regex = True)

In [34]:
# 강사가 질문에 답변했는지를 알아보고, 답변을 했다면 답변에 걸린 기간을 가져옴
instr_anwsers_list = []
no_instr_anwsers = 0

for index in range(len(temp_question_df)) :
    for element in range(len(temp_question_df['answer'].str.split(',')[index])) :
        if temp_question_df['answer'].str.split(', ')[index][element] == temp_question_df['instructor'][index] :
            instr_anwsers_list.append(temp_question_df['a_date'].str.split(', ')[index][element])
            no_instr_anwsers += 1
            break
    if no_instr_anwsers == 0 :
        instr_anwsers_list.append('None')
    no_instr_anwsers = 0

강사의 답변한 기간('a_date')은 스크래핑 날짜를 기준으로 '분 전' / '시간 전' / '일 전' / '달 전' / '2021.01.01'(1년 이상인 경우)의 형태로 표현되어 있습니다.

이를 1년 이상인 경우와 동일한 날짜 형태로 정제합니다.

In [35]:
instr_anwsers_df = pd.DataFrame(instr_anwsers_list, columns = ['a_date'])
instr_anwsers_df.columns = ['a_date']
crawling_date = datetime.today().strftime('%Y.%m.%d')

In [36]:
instr_anwsers_df[instr_anwsers_df['a_date'].str.contains('분 전')] = crawling_date
instr_anwsers_df[instr_anwsers_df['a_date'].str.contains('시간 전')] = crawling_date

In [37]:
day_index = instr_anwsers_df[instr_anwsers_df['a_date'].str.contains('일 전')].index
instr_anwsers_df['a_date'] = instr_anwsers_df['a_date'].str.replace('일 전', '')

for index in day_index :
    instr_anwsers_df.loc[index, 'a_date'] = (pd.to_datetime(crawling_date) - relativedelta(days = int(instr_anwsers_df.loc[index, 'a_date']))).strftime('%Y.%m.%d')

스크래핑 날짜에서 1달 이상 ~ 1년 미만의 데이터는 정확한 답변 기간을 알 수 없습니다.

그 이유는 질문 A가 30일 전에 게시된 질문이고 질문 B가 59일 전에 게시되었다고 가정할 때 두 질문 모두, '1달 전'으로 표기되기 때문입니다.

따라서 '달 전'의 값을 가진 데이터의 정확한 날짜를 알 수 없으므로 '달 전' 데이터를 제외한 데이터들의 답변까지 걸린 기간(강사가 답변한 날짜 - 질문이 게시된 날짜)에서 가장 많은 빈도 수를 가진 '0' (하루이내 답변함)으로 '달 전' 데이터를 처리하였습니다.

In [38]:
month_index = instr_anwsers_df[instr_anwsers_df['a_date'].str.contains('달 전')].index
instr_anwsers_df['a_date'] = instr_anwsers_df['a_date'].str.replace('달 전', '')
for index in month_index :
    instr_anwsers_df.loc[index, 'a_date'] = (pd.to_datetime(crawling_date) - relativedelta(months = int(instr_anwsers_df.loc[index, 'a_date'])) + relativedelta(days = int(instr_anwsers_df.loc[index, 'a_date']))).strftime('%Y.%m.%d')

강사가 답변한 기간 컬럼('a_date')이 모두 정제된 뒤, 질문한 날짜(q_date)를 빼서 강사가 답변까지 걸린 기간('answer_period')를 생성합니다.

In [39]:
answer_period_df = pd.concat([temp_question_df['q_date'], instr_anwsers_df['a_date']],  axis = 1)
answer_period_df['q_date'] = pd.to_datetime(answer_period_df['q_date']).dt.date

temp_answer_period_df = answer_period_df[answer_period_df['a_date'] != 'None']

temp_answer_period_df['q_date'] = pd.to_datetime(temp_answer_period_df['q_date'])
temp_answer_period_df['a_date'] = pd.to_datetime(temp_answer_period_df['a_date'])

temp_answer_period_df['answer_period'] = temp_answer_period_df['a_date'] - temp_answer_period_df['q_date']
temp_answer_period_df['answer_period'] = temp_answer_period_df['answer_period'].astype(str)
temp_answer_period_df['answer_period'] = temp_answer_period_df['answer_period'].str.replace(' days', '').astype(int)

answer_period_df['answer_period'] = temp_answer_period_df['answer_period']

In [40]:
answer_period_df = answer_period_df.drop(columns = ['q_date'])
answer_period_df = answer_period_df.rename(columns = {'a_date' : 'instr_a_date'})

question_df = pd.concat([question_df, answer_period_df],  axis = 1)
question_df.to_csv('o3_question.csv', encoding = 'utf-8-sig')

강의 게시판에 없는 강의의 질문을 삭제합니다. (현재 서비스되지 않는 강의)

In [41]:
course_list = list(course_df['course_title'])
question_df = question_df.drop(question_df[(question_df['course_title'].isin(course_list) == False) & (question_df['course_title'] != 'None')].index)

정제된 데이터를 이용하여 4개의 변수를 생성합니다.

In [42]:
question_df['has_answer'] = question_df['instr_a_date'].apply(lambda x: 0 if x == 'None' else 1)

In [43]:
total_question = question_df.groupby('course_title')['has_answer'].count()
answered = question_df.groupby('course_title')['has_answer'].sum()
answer_rate = question_df.groupby('course_title')['has_answer'].sum() / question_df.groupby('course_title')['has_answer'].count()
answer_period = question_df.groupby('course_title')['answer_period'].mean().fillna('None')

생성한 변수를 'course_df'에 추가합니다.

In [45]:
def add_question_variable(grouped_has_answer, variable_name) :
    course_title_list = list(grouped_has_answer.index)
    course_title_idx = []

    # 질문 게시글의 강의 제목과 강의 df의 강의 제목이 같은 강의 df의 index를 가져옴
    for course_title_ in course_title_list :
        if course_df[course_df['course_title'] == course_title_].index.size != 0 :
            course_title_idx.append(course_df[course_df['course_title'] == course_title_].index[0])

    # 각 위치에 맞게 전체 질문 수 변수의 값을 넣는다.
    for idx in range(len(course_title_idx)) :
        course_df.loc[course_title_idx[idx], variable_name] = grouped_has_answer[idx]
    return course_df

In [46]:
course_df = add_question_variable(total_question, 'total_question')
course_df = add_question_variable(answered, 'answered')
course_df = add_question_variable(answer_rate, 'answer_rate')
course_df = add_question_variable(answer_period, 'answer_period')

In [ ]:
course_df.to_csv('o3_course.csv', encoding = 'utf-8-sig')